# A Multiperiod Planning Example with Backlog -- ShoeCo Revisited

ShoeCo needs to plan production for the next 4 months. At the beginning of the first month, ShoeCo has 500 pairs of shoes in inventory. The demand forecast for each of the next 4 months is 

Month: |    1 |    2 |    3 |    4 |
 -----:|-----:|-----:|-----:|-----:|
Demand:| 3000 | 5000 | 2000 | 1000 |


ShoeCo has 100 workers currently employed who work 160 hours/month. Each worker is paid \\$1500/month. They can also work overtime for up to 20 hours/worker each month. Overtime is paid at \\$13/hour.  ShoeCo can choose to hire a new worker for \\$1600 or fire a worker for \\$2000. To produce a pair of shoes, 4 hours of labor and \\$15 of raw materials are required. ShoeCo must meet demand by the end of the 4 months and must pay \\$3 for every pair of shoes in inventory at the end of each month. ShoeCo must also pay \\$20 for any demand that is backlogged each month. How should ShoeCo produce for the next 4 months if they are trying to minimize their total costs?

## Model and Solution in Julia

In [1]:
using JuMP, Clp

d = [3000 5000 2000 1000] # monthly shoe demand
 
m = Model(Clp.Optimizer)

@variable(m, x[1:4] >= 0 ) # shoes produced in month t=1,2,3,4
@variable(m, w[1:4] >= 0 ) # workers employed in month t=1,2,3,4
@variable(m, o[1:4] >= 0 ) # overtime hours in month t=1,2,3,4
@variable(m, h[1:4] >= 0 ) # workers hired in month t=1,2,3,4
@variable(m, f[1:4] >= 0 ) # workers fired in month t=1,2,3,4
@variable(m, i[1:4] ) # shoes in inventory in month t=1,2,3,4 (can be negative)
@variable(m, L[1:4] >= 0 ) # shoes leftover in month t=1,2,3,4
@variable(m, S[1:4] >= 0 ) # shoes backlogged in month t=1,2,3,4

# our objective is to minimize the total cost
@objective(m, Min, 15*sum(x) + 16*sum(o) + 1600*sum(h) + 2000*sum(f)
    + 1500*sum(w) + 3*sum(L) + 20*sum(S))

# constraint on how many total hours are available for shoe production
@constraint(m, production[t in 1:4], 4*x[t] <= 160*w[t] + o[t])
# overtime cannot be greater than 20*# workers in each month
@constraint(m, overtime[t in 1:4], o[t] <= 20*w[t])

# include a constraint that requires inventory to be the difference
# between leftovers and shortage each month
@constraint(m, inv_ident[t in 1:4], i[t] == L[t] - S[t])
@constraint(m, i[4] >= 0) # enforce that all demand is met eventually
 
# INVENTORY BALANCE #
# amount we start with + amount we produce = amount we sell + amount that carries to next month

# balance inventory in the first month 
@constraint(m, inv_bal_init, 500 + x[1] == d[1] + i[1])
# balance inventory for all months after the first
@constraint(m, inv_bal[t in 2:4], i[t-1] + x[t] == d[t] + i[t])

# WORKER BALANCE #
# number currently employed - number fired + number hired = number workers available

# balance workers in first month
@constraint(m, work_bal_init, 100 - f[1] + h[1] == w[1])
# balance workers in all months after the first
@constraint(m, work_bal[t in 2:4], w[t-1] - f[t] + h[t] == w[t])

# solve this instance of ShoeCo and print relevant solution details
optimize!(m)

# note we use the Array function to turn the list of solution values into an Array.
# this helps with solution legibility.
println("Build ", Array(value.(x')), " shoes each month")
println("Use ", Array(value.(w')), " workers each month")
println("Use ", Array(value.(o')), " overtime hours each month")
println("Inventory: ", Array(value.(i')))
println("Backlog: ", Array(value.(S')))
println("Cost: ", objective_value(m))

Build [3750.0 3750.0 1500.0 1499.9999999999995] shoes each month
Use [93.75000000000001 93.75 37.5 37.49999999999999] workers each month
Use [0.0 0.0 0.0 0.0] overtime hours each month
Inventory: [1250.0 -0.0 -499.99999999999966 0.0]
Backlog: [0.0 0.0 499.99999999999966 0.0]
Cost: 690000.0
Coin0506I Presolve 16 (-5) rows, 27 (-5) columns and 51 (-8) elements
Clp0006I 0  Obj 0 Primal inf 15561.42 (5)
Clp0006I 16  Obj 690000
Clp0000I Optimal - objective value 690000
Coin0511I After Postsolve, objective 690000, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 690000 - 16 iterations time 0.022, Presolve 0.01
